In [1]:
#Alternative approach
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f4ee029617e263752d25076ce31a629f36199dce1cc3b458360d9104eaa1a9d2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import os
import sys
import pyspark
import json
import pandas as pd
import gzip
import numpy as np
import numpy as np
import re


In [3]:
#function of converting full txt to list
def extraxtString(string,startString,endString):
    startingPoint=0
    lengthString=len(string)
    myList = []
    continueYes = 1
    while(continueYes == 1 and startingPoint < lengthString):
        temp1=string.find(startString,startingPoint)
        if(temp1 != -1):
            temp1 = temp1+len(startString)
            temp2 = string.find(endString,temp1)
            if(temp2 != -1):
                middleString=string[(temp1-len(startString)):temp2]
                myList.append(middleString)
                startingPoint=temp2
            else:
                continueYes=0
        else:
            continueYes=0
    return myList



In [4]:
import pyspark.ml.classification
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
from pyspark.sql.functions import lit
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("myProject_data_intensive") \
    .getOrCreate()

In [5]:


import os

#Create an empty pyspark dataframe
schema = StructType([StructField('overall', StringType(), True),
                     StructField('reviewText', StringType(), True)])
emptyRDD = spark.sparkContext.emptyRDD()
df = spark.createDataFrame(emptyRDD,schema)
#df.printSchema()
df.show()


dataPath="/content/drive/MyDrive/Colab Notebooks/test"

columns = ["overall", "reviewText"]
start1="overall: "
end1=".0,"
start2="reviewText: "
end2=", summary"

for filename in os.listdir(dataPath):
    print(filename)
    mystring = open(os.path.join(dataPath, filename)).read().replace('\n', '')
    ListReviewers = extraxtString(mystring,"overall:","overall:")

    myList=[]
    for i in range(0,len(ListReviewers)):
        s=ListReviewers[i]
        if "overall" and "reviewText" in s:
            string1 = s[s.find(start1)+len(start1):s.rfind(end1)]
            string2 = s[s.find(start2)+len(start2):s.rfind(end2)]
            myList.append([string1,string2])

    print(filename,"is structured.")
    #dataTemp = myList[0:1000]
    dataTemp = myList
    dataFrameTemp = spark.createDataFrame(dataTemp, columns)
    dataFrameTemp.show()

    # concatinate data frames
    df = df.union(dataFrameTemp)



+-------+----------+
|overall|reviewText|
+-------+----------+
+-------+----------+

test4.txt
test4.txt is structured.
+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|      4|I've been using D...|
|      4|The demo is done ...|
|      5|If you've been wa...|
|      5|I've been creatin...|
|      5|I decided (after ...|
|      5|The video is well...|
|      5|I spent several h...|
|      5|I have had Dreamw...|
|      5|I have also taken...|
|      5|Even though I use...|
|      3|I waited to compl...|
|      5|As someone who ha...|
|      5|Adobe approved \A...|
|      5|This is a greate ...|
|      3|There are over 10...|
|      5|I am not an avid ...|
|      5|I am a long time ...|
|      4|The \Learn by Vid...|
|      4|The presenter her...|
|      4|This certified as...|
+-------+--------------------+
only showing top 20 rows

test2.txt
test2.txt is structured.
+-------+--------------------+
|overall|          reviewText|
+-------+----

In [6]:


df.show()

+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|      4|I've been using D...|
|      4|The demo is done ...|
|      5|If you've been wa...|
|      5|I've been creatin...|
|      5|I decided (after ...|
|      5|The video is well...|
|      5|I spent several h...|
|      5|I have had Dreamw...|
|      5|I have also taken...|
|      5|Even though I use...|
|      3|I waited to compl...|
|      5|As someone who ha...|
|      5|Adobe approved \A...|
|      5|This is a greate ...|
|      3|There are over 10...|
|      5|I am not an avid ...|
|      5|I am a long time ...|
|      4|The \Learn by Vid...|
|      4|The presenter her...|
|      4|This certified as...|
+-------+--------------------+
only showing top 20 rows

